## Step 1: Isolating COVID-19 Related Conditions
In this step, I created a temporary table, `covid_occurrences`, to extract and isolate all occurrences of COVID-19-related conditions based on a predefined list of `condition_concept_id`'s provided during classwork. This allows for a more focused analysis of comorbidities and treatment outcomes related to COVID-19.

### SQL Code:
```sql
-- Create a temporary table with COVID-19 related condition occurrences
create temporary table covid_occurrences as
with covid_condition_concepts as (
    select unnest(array[
        3661631,37310254,756039,37310284,3656667,3661405,3661748,
        3661632,3656668,3661408,3663281,37310283,3661885,3661406,
        37310287,3655977,756031,3662381,37310286,3655975,3656669,
        3655976,37311061
    ]) as condition_concept_id
)
select co.*
from condition_occurrence co
join covid_condition_concepts ccc on co.condition_concept_id = ccc.condition_concept_id;
```

## Step 2: Creating the `covid_comorbidities` Temporary Table

In this step, I created a temporary table named `covid_comorbidities` to capture comorbid conditions for patients diagnosed with COVID-19. This table includes conditions that were either ongoing or ended after the COVID-19 diagnosis and were diagnosed before the COVID-19 condition itself. This helps in identifying pre-existing conditions that could impact the severity or outcomes of COVID-19.

### SQL Code:
```sql
create temp table covid_comorbidities as
select
    co.person_id,
    cco.condition_concept_id,
    c.concept_name
from covid_occurrences co
join condition_occurrence cco on co.person_id = cco.person_id
join concept c on cco.condition_concept_id = c.concept_id
where (cco.condition_end_date is null 
       or cco.condition_end_date > co.condition_end_date) -- Ongoing condition or ended after COVID
  and cco.condition_start_date < co.condition_start_date -- Condition started before COVID-19 diagnosis
  and cco.condition_concept_id != co.condition_concept_id -- Exclude COVID-19 itself
order by co.person_id, cco.condition_start_date;
